In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from seleniumbase import Driver

import pandas as pd
import time
import json

# options = Options()
# options.headless = False

# driver = webdriver.Chrome(options=options,service=ChromeService(ChromeDriverManager().install()))

In [7]:
def get_match_events(driver, whoscored_match_id=1640761):
    whoscored_match_url = f'https://www.whoscored.com/Matches/{whoscored_match_id}/Live/'
    driver.get(whoscored_match_url)
    scriptdiv = driver.find_elements(By.CSS_SELECTOR, 'script')
    # indicator_string = 'require.config.params["args"]'
    indicator_string = 'matchCentreData'
    for i in range(len(scriptdiv)):
        jstring = scriptdiv[-1*(i+1)].get_attribute('innerHTML')
        if indicator_string in jstring:
            break
    jstring = jstring.replace('require.config.params["args"] = ', '')
    jstring = jstring.replace('\n', '')
    jstring = jstring.replace('            ', '')
    jstring = jstring.replace('{matchId:', '{"matchId":')
    jstring = jstring.replace('matchCentreData:', '"matchCentreData":')
    jstring = jstring.replace(
        'formationIdNameMappings', '"formationIdNameMappings"')
    jstring = jstring.replace(
        'matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    jstring = jstring.replace('};', '}')

    d = json.loads(jstring)
    matchData = d['matchCentreData']
    matchId = d['matchId']
    events = matchData['events']
    player_dict = d['matchCentreData']['playerIdNameDictionary']
    players = {
        'id': player_dict.keys(),
        'name': player_dict.values()
    }
    players_df = pd.DataFrame(players).reset_index()
    players_df['id'] = players_df['id'].astype(int)
    event_list = []
    for e in enumerate(events):
        ev = e[1]
        try:
            player_id = int(ev['playerId'])
        except:
            player_id = None
        try:
            player_name = players_df[players_df['id']
                                     == player_id]['name'].iloc[0]
            # print(f'{player_id} : {player_name}')
        except:
            player_name = None
        try:
            endX = ev['endX']
        except:
            endX = None
        try:
            endY = ev['endY']
        except:
            endY = None

        event_list.append({
            'id': ev['id'],
            'eventId': ev['eventId'],
            'minute': ev['minute'],
            'teamId': ev['teamId'],
            'playerId': player_id,
            'playerName': player_name,
            'matchId': matchId,
            'x': ev['x'],
            'y': ev['y'],
            'endX': endX,
            'endY': endY,
            'expandedMinute': ev['expandedMinute'],
            'period': ev['period']['displayName'],
            'type': ev['type']['displayName'],
            'outcomeType': ev['outcomeType']['displayName'],
            'isTouch': ev['isTouch']
        })
    event_df = pd.DataFrame(event_list)
    return event_df


In [18]:
def san_array(x):
    j = []
    for u in x:        
        if len(u):
            if "value" in u.keys():
                j.append({u["type"]["displayName"]:u["value"]})
            # else:
            #     j.append({u["type"]["displayName"]:"True"})
    return json.dumps(j)

def check_null(key, dict):
    try:
        y = dict[key]
    except:
        y = None
    return y

def get_match_events_v2(driver,match_id):
    whoscored_match_url = f'https://www.whoscored.com/Matches/{match_id}/Live/'
    driver.get(whoscored_match_url)

    scriptdiv = driver.find_elements(By.CSS_SELECTOR, 'script')

    indicator_string = 'matchCentreData'
    for i in range(len(scriptdiv)):
        jstring = scriptdiv[-1*(i+1)].get_attribute('innerHTML')
        if indicator_string in jstring:
            break
    jstring = jstring.replace('require.config.params["args"] = ', '')
    jstring = jstring.replace('\n', '')
    jstring = jstring.replace('            ', '')
    jstring = jstring.replace('{matchId:', '{"matchId":')
    jstring = jstring.replace('matchCentreData:', '"matchCentreData":')
    jstring = jstring.replace(
        'formationIdNameMappings', '"formationIdNameMappings"')
    jstring = jstring.replace(
        'matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    jstring = jstring.replace('};', '}')

    d = json.loads(jstring)
    matchData = d['matchCentreData']
    matchId = d['matchId']
    events = matchData['events']

    player_dict = d['matchCentreData']['playerIdNameDictionary']
    players = {
        'id': player_dict.keys(),
        'name': player_dict.values()
    }
    players_df = pd.DataFrame(players).reset_index()
    players_df['id'] = players_df['id'].astype(int)
    event_list = []
    for index, e in enumerate(events):
        ev = e

        try:
            player_id = int(ev['playerId'])
        except:
            player_id = None
        try:
            player_name = players_df[players_df['id'] == player_id]['name'].iloc[0]
        except:
            player_name = None
        print(player_name)

        home_id = matchData['home']['teamId']
        home_name = matchData['home']['name']
        away_id = matchData['away']['teamId']
        away_name = matchData['away']['name']

        teamName = None
        if (ev['teamId'] == home_id):
            teamName = home_name
        if (ev['teamId'] == away_id):
            teamName = away_name

        pass_recipient = None
        if index < len(events)-1:
            next_ev = events[index+1]
            if (ev['type']['displayName'] == 'Pass') & (ev['outcomeType']['displayName'] == 'Successful'):
                try:
                    pass_recipient = players_df[players_df['id'] == int(
                        next_ev['playerId'])]['name'].iloc[0]
                except:
                    pass_recipient = None

        event_list.append({
            'event_id': ev['eventId'],
            'minute': ev['minute'],
            'second': check_null('second', ev),
            'team_id': ev['teamId'],
            'x': ev['x'],
            'y': ev['y'],
            'expanded_minute': ev['expandedMinute'],
            'period': ev['period']['displayName'],
            'outcome_type': ev['outcomeType']['displayName'],
            'qualifiers': san_array(ev['qualifiers']),
            'satisfied_events_types': (ev['satisfiedEventsTypes']),
            'is_touch': ev['isTouch'],
            'player_id': player_id,
            'end_x': check_null('endX', ev),
            'end_y': check_null('endY', ev),
            'related_event_id': check_null('relatedEventId', ev),
            'related_player_id': check_null('relatedPlayerId', ev),
            'goal_mouth_z': check_null('goalMouthZ', ev),
            'goal_mouth_y': check_null('goalMouthY', ev),
            'is_shot': check_null('isShot', ev),
            'blocked_x': check_null('blockedX', ev),
            'blocked_y': check_null('blockedY', ev),
            'card_type': check_null('cardType', ev),
            'is_goal': check_null('isGoal', ev),
            'player_name': player_name,
            'event_type': ev['type']['displayName'],
            'pass_recipient': pass_recipient,
            'team_name': teamName,
            'match_date': matchData['timeStamp'],
            'match_id': matchId,
            'match_string': f"{home_name} - {away_name}"
        })

    event_df = pd.DataFrame(event_list)
    return event_df

In [19]:
driver = Driver(browser="chrome")
data = get_match_events_v2(driver,1729305)
driver.quit()

data


None
None
Mateo Kovacic
Ederson
David Raya
Gabriel Jesus
Josko Gvardiol
Jorginho
Gabriel Jesus
Nathan Aké
Nathan Aké
Ederson
William Saliba
Erling Haaland
William Saliba
Gabriel Jesus
Mateo Kovacic
Josko Gvardiol
Rico Lewis
Kyle Walker
Bernardo Silva
Rico Lewis
Bernardo Silva
Mateo Kovacic
Rúben Dias
Nathan Aké
Bernardo Silva
Nathan Aké
Rico Lewis
Rúben Dias
Nathan Aké
Gabriel Jesus
Bernardo Silva
Nathan Aké
Bernardo Silva
Rúben Dias
Ederson
Bernardo Silva
Nathan Aké
Josko Gvardiol
Phil Foden
Mateo Kovacic
Phil Foden
Rúben Dias
Kyle Walker
Rico Lewis
Bernardo Silva
Rico Lewis
Kyle Walker
Rúben Dias
Bernardo Silva
Rúben Dias
Julián Álvarez
Gabriel Magalhães
Julián Álvarez
Gabriel Magalhães
Jorginho
Jorginho
David Raya
Ben White
David Raya
Josko Gvardiol
Josko Gvardiol
Martin Ødegaard
Declan Rice
Declan Rice
Kyle Walker
Declan Rice
Declan Rice
Kyle Walker
Phil Foden
Josko Gvardiol
Declan Rice
Gabriel Jesus
Julián Álvarez
Erling Haaland
Gabriel Magalhães
Erling Haaland
Nathan Aké
Gabriel 

,event_id,minute,second,team_id,x,y,expanded_minute,period,outcome_type,qualifiers,...,blocked_y,card_type,is_goal,player_name,event_type,pass_recipient,team_name,match_date,match_id,match_string
0,2,0,0.0,13,0.0,0.0,0,FirstHalf,Successful,[],...,NaN,None,None,None,Start,None,Arsenal,2023-10-09 03:09:24,1729305,Arsenal - Man City
1,2,0,0.0,167,0.0,0.0,0,FirstHalf,Successful,[],...,NaN,None,None,None,Start,None,Man City,2023-10-09 03:09:24,1729305,Arsenal - Man City
2,3,0,0.0,167,49.7,49.9,0,FirstHalf,Successful,"[{""Angle"": ""3.31""}, {""Length"": ""29.1""}, {""Pass...",...,NaN,None,None,Mateo Kovacic,Pass,Ederson,Man City,2023-10-09 03:09:24,1729305,Arsenal - Man City
3,4,0,5.0,167,24.1,44.0,0,FirstHalf,Unsuccessful,"[{""Zone"": ""Right""}, {""Length"": ""84.9""}, {""Angl...",...,NaN,None,None,Ederson,Pass,None,Man City,2023-10-09 03:09:24,1729305,Arsenal - Man City
4,4,0,40.0,13,5.1,46.7,0,FirstHalf,Successful,"[{""PassEndY"": ""2.8""}, {""Length"": ""63.0""}, {""Zo...",...,NaN,None,None,David Raya,Pass,Gabriel Jesus,Arsenal,2023-10-09 03:09:24,1729305,Arsenal - Man City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,829,94,7.0,167,0.0,0.0,96,SecondHalf,Successful,[],...,NaN,None,None,None,End,None,Man City,2023-10-09 03:09:24,1729305,Arsenal - Man City
1543,828,0,0.0,13,0.0,0.0,6,PostGame,Successful,[],...,NaN,None,None,None,End,None,Arsenal,2023-10-09 03:09:24,1729305,Arsenal - Man City
1544,830,0,0.0,167,0.0,0.0,6,PostGame,Successful,[],...,NaN,None,None,None,End,None,Man City,2023-10-09 03:09:24,1729305,Arsenal - Man City
1545,1,0,0.0,167,0.0,0.0,0,PreMatch,Successful,"[{""PlayerPosition"": ""1, 2, 2, 3, 2, 2, 3, 3, 4...",...,NaN,None,None,None,FormationSet,None,Man City,2023-10-09 03:09:24,1729305,Arsenal - Man City
